In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
class GoogleCrawlerSelenium():
    def __init__(self,limit):
        self.links_lista = []
        self.texto_lista = []
        self.limit = limit
        self.df = None

    # Busca pelo site google
    def searchInDrive(self,query):
        driver = webdriver.Firefox(executable_path="./geckodriver")
        driver.get("https://google.com.br/")
        
        elem = driver.find_element_by_name("q") # Busca pelo botão de pesquisa
        elem.clear()
        elem.send_keys(query) # Busca a ser realizada no site
        time.sleep(5)
        driver.find_element_by_class_name("gNO89b").click() # Acionar o bota da busca

        
        for i in range(self.limit):
            time.sleep(5)
            links = driver.find_elements_by_xpath("//div[contains(@class,'yuRUbf')]")
            for link in links:
                self.links_lista.append(link.find_element_by_xpath('.//a').get_attribute('href'))
                self.texto_lista.append(link.find_element_by_xpath('.//a/h3/span').text)
            try:
                driver.find_element_by_id("pnnext").click()
            except:
                self.df = pd.DataFrame({"links" : self.links_lista, "titulo" : self.texto_lista})
                print("Numero excedido e dataframe criado")
                break
        
        driver.close() # Fecha o drive após finalizar tudo


In [3]:
crawler = GoogleCrawlerSelenium(30)
crawler.searchInDrive("coronavirus perguntas e respostas")

Numero excedido e dataframe criado


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=43085): Max retries exceeded with url: /session/5155de38-1f4e-4c42-83f7-190e5b604926/window (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7f7d7079d340>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [4]:
df = crawler.df
df.to_csv('./scraped_links.csv',index = False)

In [5]:
df

,links,titulo
0,https://coronavirus.saude.gov.br/index.php/per...,Perguntas e respostas - Coronavírus - Ministér...
1,https://portal.fiocruz.br/coronavirus/pergunta...,Coronavírus: Perguntas e respostas - Fundação ...
2,https://www.unimed.coop.br/viver-bem/saude-em-...,Coronavírus e COVID-19: perguntas e respostas ...
3,https://www.msdmanuals.com/pt-pt/profissional/...,PERGUNTAS FREQUENTES SOBRE A COVID-19 - Manuai...
4,https://www.saude.ma.gov.br/perguntas-e-respos...,Perguntas e respostas sobre coronavírus (COVID...
...,...,...
253,https://ijnet.org/pt-br/story/como-fazer-pergu...,Como fazer perguntas difíceis sobre COVID-19 |...
254,https://noticias.r7.com/minas-gerais/mg-record...,Infectologista responde perguntas sobre a covi...
255,http://www.oieie.tku.edu.tw/zh_tw/news/Oversea...,COVID 19 Lo que necesitas saber | 淡江大學 國際暨兩岸事務處
256,https://cultura.id/corona-faq-covid19,FAQ Corona: Tanya Jawab Seputar COVID-19 - Cul...


# Data análise

In [15]:
df_crawler = pd.read_csv('./scraped_links.csv')
df_reposi = pd.read_csv('./links_repositorio.csv')

In [16]:
df_reposi.dropna(inplace = True)

In [17]:
sum(df_crawler.duplicated() == True)

0

In [18]:
df_crawler.drop_duplicates(inplace = True)

In [19]:
sum(df_crawler.duplicated() == True)

0

In [20]:
aux = []
for link_repo in df_reposi.link:
    if(link_repo in df_crawler.links.values):
        aux.append(link_repo)

removed_links = []    
for i in range(len(df_crawler)):
    if(df_crawler.links.values[i] in aux):
        removed_links.append(i)
df_crawler.drop(df_crawler.index[[removed_links]],inplace=True)       

/home/p1ntsu/anaconda3/envs/crawler/lib/python3.8/site-packages/pandas/core/indexes/base.py:4111: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


In [21]:
df_crawler.reset_index(inplace=True)
del df_crawler["index"]

df_crawler

,links,titulo
0,https://www.unimed.coop.br/viver-bem/saude-em-...,Coronavírus e COVID-19: perguntas e respostas ...
1,https://www.msdmanuals.com/pt-pt/profissional/...,PERGUNTAS FREQUENTES SOBRE A COVID-19 - Manuai...
2,https://medprev.online/blog/doencas/perguntas-...,Perguntas e respostas sobre o coronavírus: tir...
3,http://telessaude.ba.gov.br/category/perguntas...,Perguntas Frequentes COVID 19 | Telessaúde Bahia
4,https://vidasaudavel.einstein.br/coronavirus/c...,Novo coronavírus (COVID-19): perguntas e respo...
...,...,...
240,https://ijnet.org/pt-br/story/como-fazer-pergu...,Como fazer perguntas difíceis sobre COVID-19 |...
241,https://noticias.r7.com/minas-gerais/mg-record...,Infectologista responde perguntas sobre a covi...
242,http://www.oieie.tku.edu.tw/zh_tw/news/Oversea...,COVID 19 Lo que necesitas saber | 淡江大學 國際暨兩岸事務處
243,https://cultura.id/corona-faq-covid19,FAQ Corona: Tanya Jawab Seputar COVID-19 - Cul...


In [22]:
for i in df_crawler.links:
    if("gov" in i):
        print(i)

http://telessaude.ba.gov.br/category/perguntas-frequentes-covid-19/
https://www.gov.br/economia/pt-br/acesso-a-informacao/perguntas-frequentes/covid-19
https://www.epa.gov/lep/perguntas-frequentes-sobre-o-coronavirus-covid-19
https://linhares.es.gov.br/perguntas-frequentes-coronavirus-covid-19/
https://www.inca.gov.br/perguntas-frequentes/cancer-e-coronavirus-covid-19
https://www.mangaratiba.rj.gov.br/coronavirus/perguntas-frequentes.html
http://www.aids.gov.br/pt-br/pub/2020/perguntas-e-respostas-frequentes-sobre-covid-19-e-tuberculose-para-pessoas-em-tratamento-da
http://www.aids.gov.br/pt-br/pub/2020/perguntas-e-respostas-sobre-covid-19-e-pessoas-em-tratamento-de-hanseniase
https://alenquer.pa.gov.br/covid19/perguntas-frequentes/
https://covid19.ins.gov.mz/f-a-q/
https://educacao.sme.prefeitura.sp.gov.br/coronavirus/perguntas-e-respostas-covid-19/
https://www.fnde.gov.br/index.php/programas/pnae/pnae-perguntas-frequentes
https://www.coren-df.gov.br/site/coren-df-lanca-manual-de-perg

In [17]:
print(df_crawler.links.values)

['https://www.unimed.coop.br/viver-bem/saude-em-pauta/coronavirus-e-covid-19-perguntas-e-respostas'
 'https://medprev.online/blog/doencas/perguntas-e-respostas-sobre-coronavrus-tire-todas-as-dvidas-sobre-pandemia/'
 'http://portal.anvisa.gov.br/noticias/-/asset_publisher/FXrpx9qY7FbU/content/testes-para-covid-19-perguntas-e-respost-1/219201'
 'https://coronavirus.saude.gov.br/sobre-a-doenca'
 'https://vidasaudavel.einstein.br/coronavirus/covid-19-faq/'
 'https://crianca.mppr.mp.br/arquivos/File/legis/covid19/cartilha_coronavirus_unimed.pdf'
 'http://coronavirus.saude.mg.gov.br/perguntaserespostas'
 'https://www.novartis.com.br/perguntas-e-respostas-sobre-covid-19'
 'http://saudedebate.com.br/noticias/perguntas-e-respostas-sobre-novo-coronavirus-e-covid-19'
 'https://www.hospitalsaodomingos.com.br/coronavirus/noticia/perguntas-e-respostas-tire-suas-as-duvidas-sobre-o-novo-coronavirus-e-a-covid-19-736'
 'https://www.prefeitura.sp.gov.br/cidade/secretarias/saude/vigilancia_em_saude/doenca